<a href="https://colab.research.google.com/github/VishanOberoi/LLMs/blob/main/FineTuningFunctions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Imports and Installs

!pip install jsonlines
!pip install datasets
! pip install -U accelerate
! pip install -U transformers
!pip install datasets



import pandas as pd
from pprint import pprint
import itertools
import jsonlines
from datasets import load_dataset
from pprint import pprint
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import datasets
import tempfile
import logging
import random
# import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd
import jsonlines
from transformers import Trainer
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
logger = logging.getLogger(__name__)
global_config = None





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 22.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
from google.colab import drive
import pandas as pd
from transformers import AutoTokenizer
from datasets import load_dataset
import torch

# Function to prepare the entire dataset
def prepare_dataset(filename, model_name, prompt_template, modified_filename, test_size=0.1, seed=123):
    """
    Prepares the dataset for training, including loading, processing, tokenizing, and splitting.
    Args:
    - filename (str): Path to the JSONL file.
    - model_name (str): Model name for the tokenizer.
    - prompt_template (str): Template for formatting the dataset.
    - modified_filename (str): Filename to save the modified dataset.
    - test_size (float): Fraction of the dataset to be used as the test set.
    - seed (int): Random seed for splitting the dataset.

    Returns:
    - Tuple[Dataset, Dataset]: Train and test datasets.
    """
    # Mount Google Drive
    drive.mount('/content/drive')

    # Load and prepare dataset
    df = pd.read_json(filename, lines=True)
    df = df.rename(columns={'Questions': 'instruction', 'Answers': 'output'})
    examples = df.to_dict()

    # Prepare finetuning dataset
    num_examples = len(examples["instruction"])
    finetuning_dataset = []
    for i in range(num_examples):
        question = examples["instruction"][i]
        answer = examples["output"][i]
        text_with_prompt_template = prompt_template.format(question=question)
        finetuning_dataset.append({"question": text_with_prompt_template, "answer": answer})

    # Save modified dataset
    df.to_json(modified_filename, orient='records', lines=True)

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.truncation_side = "left"

    # Internal tokenize function
    def tokenize_function(examples):
        texts = [instr + out for instr, out in zip(examples["instruction"], examples["output"])]
        tokenized_inputs = tokenizer(
            texts,
            padding="max_length",
            truncation=True,
            max_length=2048,
            return_tensors="np"
        )
        return tokenized_inputs

    # Load and tokenize dataset
    finetuning_dataset_loaded = load_dataset("json", data_files=modified_filename, split="train")
    tokenized_dataset = finetuning_dataset_loaded.map(
        tokenize_function,
        batched=True,
    )

    # Add labels if 'input_ids' are in the dataset
    if "input_ids" in tokenized_dataset.column_names:
        tokenized_dataset = tokenized_dataset.add_column("labels", tokenized_dataset["input_ids"])
    else:
        print("Error: 'input_ids' not found in the dataset")

    # Split dataset
    split_dataset = tokenized_dataset.train_test_split(test_size=test_size, shuffle=True, seed=seed)
    return split_dataset['train'], split_dataset['test']


In [ ]:
import torch
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from logging import getLogger

# Function to setup device for training
def setup_device():
    """
    Sets up and returns the device (GPU or CPU) for training.

    Returns:
    - device: The selected device (CUDA or CPU).
    """
    logger = getLogger()
    device_count = torch.cuda.device_count()
    if device_count > 0:
        logger.debug("Select GPU device")
        return torch.device("cuda")
    else:
        logger.debug("Select CPU device")
        return torch.device("cpu")

# Function to initialize training
def initialize_training(model_name, trained_model_name, num_train_epochs, learning_rate=1.0e-5):
    """
    Initializes the model and training arguments.

    Args:
    - model_name (str): The name of the model to be trained.
    - trained_model_name (str): The name for saving the trained model.
    - num_train_epochs (int): The number of epochs for training.
    - learning_rate (float): The learning rate for training.

    Returns:
    - model: The initialized model.
    - TrainingArguments: The training arguments.
    """
    base_model = AutoModelForCausalLM.from_pretrained(model_name)
    training_args = TrainingArguments(
        learning_rate=learning_rate,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=1,
        output_dir=trained_model_name,
        overwrite_output_dir=False,
        disable_tqdm=False,
        eval_steps=120,
        save_steps=120,
        warmup_steps=1,
        per_device_eval_batch_size=1,
        evaluation_strategy="steps",
        logging_strategy="steps",
        logging_steps=1,
        optim="adafactor",
        gradient_accumulation_steps=4,
        gradient_checkpointing=False,
        load_best_model_at_end=True,
        save_total_limit=1,
        metric_for_best_model="eval_loss",
        greater_is_better=False
    )
    return base_model, training_args

# Function to train the model
def train_model(base_model, training_args, train_dataset, test_dataset):
    """
    Trains the model using the provided datasets.

    Args:
    - base_model: The model to be trained.
    - training_args: The training arguments.
    - train_dataset: The training dataset.
    - test_dataset: The testing dataset.

    Returns:
    - training_output: The result of the training process.
    """
    trainer = Trainer(
        model=base_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset
    )
    return trainer.train()

# Existing inference function (unchanged)
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [ ]:
def save_model(trainer, output_dir):
    """
    Saves the trained model to the specified directory.

    Args:
    - trainer: The Trainer object used for training the model.
    - output_dir (str): The directory where the model should be saved.

    Returns:
    - save_dir (str): The directory where the model was saved.
    """
    save_dir = f'{output_dir}/final'
    trainer.save_model(save_dir)
    print("Saved model to:", save_dir)
    return save_dir

# Function to test the finetuned model
def test_model(save_dir, device, test_dataset, tokenizer):
    """
    Tests the finetuned model on all samples from the test dataset.

    Args:
    - save_dir (str): The directory where the finetuned model is saved.
    - device: The device (CPU or CUDA) on which the model runs.
    - test_dataset: The dataset used for testing.
    - tokenizer: The tokenizer used for the model.
    """
    finetuned_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)
    finetuned_model.to(device)

    for i, sample in enumerate(test_dataset):
        test_question = sample['instruction']
        test_answer = sample['output']

        print(f"Question input (test {i+1}):", test_question)

        finetuned_answer = inference(test_question, finetuned_model, tokenizer)
        print("Finetuned model's answer:", finetuned_answer)

        print("Target answer output:", test_answer)
        print("-------------------------------------------------")




In [ ]:
class LLMFineTuner:
    def __init__(self, model_name, output_dir, filename, modified_filename, prompt_template):
        self.model_name = model_name
        self.output_dir = output_dir
        self.filename = filename
        self.modified_filename = modified_filename
        self.prompt_template = prompt_template
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        self.device = setup_device()  # setup_device function as defined earlier

    def prepare_and_tokenize(self, test_size=0.1, seed=123):
        self.train_dataset, self.test_dataset = prepare_dataset(
            filename=self.filename,
            model_name=self.model_name,
            prompt_template=self.prompt_template,
            modified_filename=self.modified_filename,
            test_size=test_size,
            seed=seed
        )  # prepare_dataset function as defined earlier


    def test_custom(self, custom_question, max_input_tokens=1000, max_output_tokens=100):
        if self.model is None:
            print("Model not initialized or trained. Please train the model first.")
            return None

        # Generate answer for the custom question
        answer = inference(
            text=custom_question,
            model=self.model,
            tokenizer=self.tokenizer,
            max_input_tokens=max_input_tokens,
            max_output_tokens=max_output_tokens
        )

        return answer

    def initialize_and_train(self, trained_model_name, num_train_epochs, learning_rate=1.0e-5):
        self.model, self.training_args = initialize_training(
            model_name=self.model_name,
            trained_model_name=trained_model_name,
            num_train_epochs=num_train_epochs,
            learning_rate=learning_rate
        )
        # Store the Trainer object as an attribute
        self.trainer = Trainer(
            model=self.model,
            args=self.training_args,
            train_dataset=self.train_dataset,
            eval_dataset=self.test_dataset
        )
        self.training_output = self.trainer.train()


    def save_and_test(self):
        # Use the stored Trainer object instead of the TrainOutput
        self.save_dir = save_model(
            trainer=self.trainer,  # Updated to use the Trainer object
            output_dir=self.output_dir
        )
        test_model(
            save_dir=self.save_dir,
            device=self.device,
            test_dataset=self.test_dataset,
            tokenizer=self.tokenizer
        )

    def generate_inference(self, text, max_input_tokens=1000, max_output_tokens=100):
        return inference(
            text=text,
            model=self.model,
            tokenizer=self.tokenizer,
            max_input_tokens=max_input_tokens,
            max_output_tokens=max_output_tokens
        )  # inference function as defined earlier


In [ ]:
# Define your parameters
filename = '/content/drive/MyDrive/Data/data27.jsonl'
modified_filename = '/content/drive/MyDrive/Data/modified_data27.json'
model_name = "EleutherAI/pythia-70m"
output_dir = "maitribot_output"  # Define a directory name for output
prompt_template = """Below is a question. Write a response that appropriately answers it and use as much detail as you can. ### Question:\n{question}\n\n### Answer:"""
max_steps = 3  # Define max_steps (if it's used in your training configuration)
num_train_epochs = 10  # Define num_train_epochs
trained_model_name = f"maitribot_{max_steps}_steps_{num_train_epochs}_epochs"

# Instantiate the class
finetuner = LLMFineTuner(model_name, output_dir, filename, modified_filename, prompt_template)

# Prepare and tokenize the dataset
finetuner.prepare_and_tokenize()

# Initialize and train the model
finetuner.initialize_and_train(trained_model_name, num_train_epochs)

# Save the model and test
finetuner.save_and_test()





/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

...

Saved model to: maitribot_output/final


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test 1): What makes Maitri lab-grown diamonds ethical and sustainable?
Finetuned model's answer: Maitri lab-grown diamonds are designed to provide a unique environment for natural beauty and beauty.
Target answer output: Maitri lab-grown diamonds are considered more sustainable as they require fewer environmental resources and do not involve unethical mining practices.
-------------------------------------------------
Question input (test 2): What is the market trend for Maitri lab-grown diamonds?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Finetuned model's answer: Maitri lab-grown diamonds are typically priced lower than traditional diamonds, and their natural beauty is often evaluated using traditional diamonds.
Target answer output: The market for Maitri lab-grown diamonds is growing steadily as consumers increasingly value ethical and sustainable choices in their purchases.
-------------------------------------------------
Question input (test 3): Can customers request specific cut styles for their CVD diamond?
Finetuned model's answer: Yes, cut styles are often cut into cut diamonds using cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut cut
Target answer output: Absolutely, we offer customization options, and customers can request specific cut styles for their C

In [ ]:
custom_question = "Is Matri AI certified"
custom_answer = finetuner.test_custom(custom_question)
print(f"Custom Question: {custom_question}")
print("Model's answer:", custom_answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Custom Question: Is Matri AI certified
Model's answer: ?Yes, Maitri lab-grown diamonds are certified by Maitri lab-grown diamonds.


In [ ]:
custom_question = "Can you customize maitri diamonds"
custom_answer = finetuner.test_custom(custom_question)
print(f"Custom Question: {custom_question}")
print("Model's answer:", custom_answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Custom Question: Can you customize maitri diamonds
Model's answer: ?Yes, we can customize your purchase to make sure you're happy.


In [ ]:
custom_question = "What do cut grades mean in maitriAI diamonds?"
custom_answer = finetuner.test_custom(custom_question)
print(f"Custom Question: {custom_question}")
print("Model's answer:", custom_answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Custom Question: What do cut grades mean in maitriAI diamonds?
Model's answer: A diamond cut diamond cut into diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds, diamonds,


In [ ]:
custom_question = "What is the return policy"
custom_answer = finetuner.test_custom(custom_question)
print(f"Custom Question: {custom_question}")
print("Model's answer:", custom_answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Custom Question: What is the return policy
Model's answer:  for Maitri lab-grown diamonds?The return policy for Maitri lab-grown diamonds is based on the return policy for Maitri lab-grown diamonds.


In [ ]:
custom_question = "What effect does clarity have in the CVD diamond"
custom_answer = finetuner.test_custom(custom_question)
print(f"Custom Question: {custom_question}")
print("Model's answer:", custom_answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Custom Question: What effect does clarity have in the CVD diamond
Model's answer: , diamond and diamond diamonds are diamonds.


In [ ]:
# Define your parameters
filename = '/content/drive/MyDrive/Data/data27.jsonl'
modified_filename = '/content/drive/MyDrive/Data/modified_data27.json'
model_name = "EleutherAI/pythia-160m"
output_dir = "maitribot_output"  # Define a directory name for output
prompt_template = """Below is a question. Write a response that appropriately answers it and use as much detail as you can. ### Question:\n{question}\n\n### Answer:"""
max_steps = 3  # Define max_steps (if it's used in your training configuration)
num_train_epochs = 10  # Define num_train_epochs
trained_model_name = f"maitribot_{max_steps}_steps_{num_train_epochs}_epochs"

# Instantiate the class
finetuner = LLMFineTuner(model_name, output_dir, filename, modified_filename, prompt_template)

# Prepare and tokenize the dataset
finetuner.prepare_and_tokenize()

# Initialize and train the model
finetuner.initialize_and_train(trained_model_name, num_train_epochs)

# Save the model and test
finetuner.save_and_test()





/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Question input (test 1): What makes Maitri lab-grown diamonds ethical and sustainable?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Finetuned model's answer: Maitri lab-grown diamonds are diamonds created in a controlled laboratory environment using advanced technological processes.
Target answer output: Maitri lab-grown diamonds are considered more sustainable as they require fewer environmental resources and do not involve unethical mining practices.
-------------------------------------------------
Question input (test 2): What is the market trend for Maitri lab-grown diamonds?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Finetuned model's answer: Maitri lab-grown diamonds are emerging as valuable assets for emerging markets such as emerging-southeast Asia, the United States, and Latin America.
Target answer output: The market for Maitri lab-grown diamonds is growing steadily as consumers increasingly value ethical and sustainable choices in their purchases.
-------------------------------------------------
Question input (test 3): Can customers request specific cut styles for their CVD diamond?
Finetuned model's answer: Specific cut styles are available for CVD diamonds from the Diamond Processing and Maitri lab to ensure customer satisfaction.
Target answer output: Absolutely, we offer customization options, and customers can request specific cut styles for their CVD diamonds to suit their preferences.
-------------------------------------------------


In [ ]:
custom_question = "What is a diamond"
custom_answer = finetuner.test_custom(custom_question)
print(f"Custom Question: {custom_question}")
print("Model's answer:", custom_answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Custom Question: What is a diamond
Model's answer:  cut diamond in diamond parlance?It refers to diamonds cut diamonds in diamond parlance. Diamonds cut diamonds are diamonds that have been cut into pieces of varying sizes and shapes.
